In [2]:
%pylab notebook

Populating the interactive namespace from numpy and matplotlib


# 4.2.  probabilistic Generative Models
## Derivation of the Softmax

Some notation:
$\require{cancel}$
$\def\x{\mathbf{x}}$
$\def\maha{\Delta}$
$\def\w{\mathbf{w}}$
$\def\X{\mathbf{X}}$

- let $\w$ denote a vector of weights; $w_0$ is a bias parameter and $\tilde\w=(w_0,\w)$ is the concatenated weights, where the ~ is omitted when clear. 
- Let $\maha^2$ denote the squared mahanalobis distance (the part in the exponent of a Gaussian)
- Let $\x$ denote an arbitraty input sample, $\x_n$ is the n'th sample in a set, $\X$ is the entire training set. 
- $K$ is the number of classes, $D$ is the dimensionality of $\x$. 

We know _Bayes_ theorem that 
$$ p(C_k | \x) = \frac{p(\x | C_k) p (C_k)} {p(\x)}$$

### Probabilitic Generative Model

Since the we are going to estimate the _likelihoods_ $p(x|C_k)$ and the priors $p(C_k)$, it is possible to estimate the probability of each sample $p(x)$. 

$$p(\x) = \sum_{j=1}^K P(\x|C_j)p(C_j)$$

In principle, since we know $p(\x)$ it is possible to generate input data sets that are similar, so we call this as **generative** model.

### Discriminative Model
In principle, one can classify samples as long as you can partition the input region $x$ into regions $R_1, R_2, ... R_k$ so that $x\in R_k$ means we decide to give $x$ the label $k$. Knowing the regions $R_k$ is not sifficient to generate new plausible samples. 

The probabilitic generative models are also discriminative, but they have more information about the data. 

### Deriving Softmax

$$\begin{align}
k_\text{pred} &= \arg\max_k p(C_k|x)\\
p(C_k|x) &= \frac{p(x | C_k) p (C_k)} {p(x)} & \text{(Bayes)}\\
  &=  \frac{p(x | C_k) p (C_k)} {\sum_j p(x | C_j) p (C_j)} & \text{(Sum Rule)}\\
  &= \frac{\exp\{a_k\}} {\sum_j \exp\{a_j\}}, & a_k = \ln\{p(x|C_k)p(C_k)\}
\end{align}$$
So the softmax is just rewriting Bayes therem, so that we can focus on the log-space value $a_k$. This is useful because probabilities of independant events are multiplied, which is numerically unstable. It is more convenient to use logarithm because products become sums.

> **NOTE:** Your book uses the variable $a_k$ for 'activation', which is a term related to neural networks. 

### Deriviing the sigmoid function for binary classification
We have the softmax
$$\begin{align}
p(C_k|x)  &= \frac{\exp\{a_k\}} {\sum_j \exp\{a_j\}}, & a_k = \ln\{p(x|C_k)p(C_k)\}\\
p(C_1|x) &=  \frac{\exp\{a_1\}} {\exp\{a_1\}+\exp\{a_2\}} & \text{(For $K=2$))}\\
&=  \frac{1} {1+\exp\{a_2\}/\exp\{a_1\}} & \text{reduce the fraction}\\
&=  \frac{1} {1+\exp\{a_2-a_1\}} & \text{combine exponents}\\
&=  \frac{1} {1+\exp\{-(a_1-a_2)\}} & \text{}\\
&=  \frac{1} {1+\exp\{-a\}} & a=\ln\frac{p(x | C_1) p (C_1)}{p(x | C_2) p (C_2)}\\
&= \sigma(a)
\end{align}$$
which is the sigmoid function, useful for binary classification. 

### Logistic Regression (Multinomial)
$\def\norm{\mathcal{N}}$
Let's make some **assumptions**:
 - Each class $C_k$ is normally distributed data; that is, $p(x|C_k) = \norm(x|\mu_k, \Sigma)$.
 - All classes share the _**same**_ covariance, $\Sigma$.
 
Our **goal** here is to find definitions for $a_k$ in the definition of a sigmoid for multinomial classification.  In particular, we will represent each $a_k$ as a _linear_ function of $\x$ so for _some_ choice of $\w_k, w_{k0}$ we will find $a_k = \w_k^T\x+w_{k0}$.  It turns out that, given our assumptions above, things will work out nicely for us. 

$$\begin{align}
p(x|C_1)  &=  \frac{p(x | C_k) p (C_k)} {\sum_j p(x | C_j) p (C_j)} & \text{(see above)}\\
&= \frac{\norm(x|\mu_1,\Sigma)p(C_1)}{\sum_j \norm(x|\mu_j,\Sigma)p(C_j)} &\text{Gaussian asumption}\\
&= \frac{
\cancel{(2\pi)^{-D/2}|\Sigma|^\frac{1}{2}}\exp\{\maha^2_1)\}p(C_1)}{\sum_j \cancel{(2\pi)^{-D/2}|\Sigma|^\frac{1}{2}}\exp\{\maha^2_j\}p(C_j) } & \text{where $\Delta_j$ is mahanlobis dist.}\\
&= \frac{
\exp\{-\frac{1}{2}(\x-\mu_1)^T\Sigma^{-1}(\x-\,u_1)\}p(C_1)}{\sum_j \exp\{\maha^2_j\}p(C_j) } & \text{subst. $\Delta^2_1$}\\
&= \frac{
\exp\{-\frac{1}{2}\x^T\Sigma^{-1}\x + \mu_1^T\Sigma^{-1}\x  -\frac{1}{2}\mu_1^T\Sigma^{-1}\mu_1\}p(C_1)}{\sum_j \exp\{\maha^2_j\}p(C_j) } & \text{expand}\\
&= \frac{
\exp\{-\frac{1}{2}\x^T\Sigma^{-1}\x\}\exp\{\mu_1^T\Sigma^{-1}\x  -\frac{1}{2}\mu_1^T\Sigma^{-1}\mu_1\}p(C_1)}{\sum_j \exp\{\maha^2_j\}p(C_j) } & \text{Split the exponent}\\
&= \frac{
\cancel{\exp\{-\frac{1}{2}\x^T\Sigma^{-1}\x\}}\exp\{\mu_1^T\Sigma^{-1}\x  -\frac{1}{2}\mu_1^T\Sigma^{-1}\mu_1\}p(C_1)}
{\sum_j \cancel{\exp\{-\frac{1}{2}\x^T\Sigma^{-1}\x\}}\exp\{\mu_j^T\Sigma^{-1}\x  -\frac{1}{2}\mu_j^T\Sigma^{-1}\mu_j\}p(C_j) } & \text{ditto for the denom.}\\
&= \frac{
\exp\{\mu_1^T\Sigma^{-1}\x  -\frac{1}{2}\mu_1^T\Sigma^{-1}\mu_1\}p(C_1)}
{\sum_j \exp\{\mu_j^T\Sigma^{-1}\x  -\frac{1}{2}\mu_j^T\Sigma^{-1}\mu_j\}p(C_j) } & \text{}\\
&= \frac{
\exp\{\mu_1^T\Sigma^{-1}\x  -\frac{1}{2}\mu_1^T\Sigma^{-1}\mu_1 + \ln p(C_1)\}}
{\sum_j \exp\{\mu_j^T\Sigma^{-1}\x  -\frac{1}{2}\mu_j^T\Sigma^{-1}\mu_j +\ln p(C_j)\} } & \text{combine exponents ($\exp(\ln ..)$)}\\
&= \frac{
\exp\{(\Sigma^{-1}\mu_1)^T\x  -\frac{1}{2}\mu_1^T\Sigma^{-1}\mu_1 + \ln p(C_1)\}}
{\sum_j \exp\{\underbrace{(\Sigma^{-1}\mu_j)^T}_{\w_j^T}\x  \underbrace{-\frac{1}{2}\mu_j^T\Sigma^{-1}\mu_j +\ln p(C_j)}_{w_{j0}}\} } & \text{play with transpose}\\
&= \frac{
\exp\{\w_1^T\x + w_{10}\}}
{\sum_j \exp\{\w_j^T\x + w_{j0}\} } & \text{subst. $\w_j, w_{j0}$}\\
&= \frac{
\exp\{a_1\}}
{\sum_j \exp\{a_j\} } & \text{subst. $a_j=\w_j^T\x + w_{j0}$}\\
\end{align}$$

Hidden in there is my definition for $\w_j$ and $w_{j0}$, 
$$\w_j=\Sigma^{-1}\mu_j,  \hspace{4em}w_{j0} = -\frac{1}{2}\mu_j^T\Sigma^{-1}\mu_j + \ln p(C_j)$$

Because we choose the minimim error (max posterior) classification, the regions $\{R_k\}$ that result from setting $k_\text{pred}=\arg\max_k p(C_k|x)$ are identical to the regions you would find by setting $k_\text{pred}=\arg\max_k a_k$. 

## Logistic Regression (Binary)
This is actually the _most common_ version of logistic regression. 

Our assumptions are the same as above, with the addition of a binary class assumption:
 - Each class $C_k$ is normally distributes data; that is, $p(x|C_k) = \norm(x|\mu_k, \Sigma)$.
 - All classes share the _**same**_ covariance, $\Sigma$.
 - The number of classes ($K=2$), i.e. we are doing binary classification. 
 

For binary problems we have found

$$\begin{align}
p(C_1|x) &= \sigma(a) \\
         &= \frac{1}{1+\exp\{-a\}}
\end{align}$$
where 
$$\begin{align}
a &= a_1-a_2 \\
  &= \w_1^T\x - \w_2^T\x + w_{10} - w_{20} \\
  &= \underbrace{(\w_1 - \w_2)}_{\w}^T\x + \underbrace{(w_{10} - w_{20})}_{w_0}\\
  &= \w^T\x+w_0\\
\end{align}$$

so 
$$\begin{align}
\w &= \w_1-\w_2 \\
  &= \Sigma^{-1}\mu_1 - \Sigma^{-1}\mu_2 \\
  &= \Sigma^{-1}(\mu_1 - \mu_2)\\
w_0 &= w_{10} - w_{20} \\
    &= \underbrace{ -\frac{1}{2}\mu_1^T\Sigma^{-1}\mu_1 + \ln p(C_1)}_{w_{10}} 
       \underbrace{ +\frac{1}{2}\mu_2^T\Sigma^{-1}\mu_2 - \ln p(C_2)}_{-w_{20}}\\
    &= \frac{1}{2}(\mu_2^T\Sigma^{-1}\mu_2-\mu_1^T\Sigma^{-1}\mu_1) + \ln p(C_1)-\ln p(C_2)\\
    &= \frac{1}{2}(\mu_2-\mu_1)^T\Sigma^{-1}(\mu_2-\mu_1) + \ln p(C_1)-\ln p(C_2)
\end{align}$$